# MSPR : e-amazing

## 0. En Amont filtrage de tous les utilisateurs qui se terminent par 1 pour alléger les exécutions par la suite

In [ ]:
# file_list = [
#     'data/2019-Oct.csv',
#     'data/2019-Nov.csv',
#     'data/2019-Dec.csv',
#     'data/2020-Jan.csv',
#     'data/2020-Feb.csv',
#     'data/2020-Mar.csv',
#     'data/2020-Apr.csv'
# ]

In [ ]:
f_list = [spark.read.csv(file, header=True, inferSchema=True) for file in file_list]

# Combiner les DataFrames en un seul
combined_df = df_list[0]
for df in df_list[1:]:
    combined_df = combined_df.union(df)

# Filtrer les données où user_id se termine par '1'
filtered_df = combined_df.filter(combined_df['user_id'].cast("string").endswith('1'))



# Spécifiez le chemin où vous voulez sauvegarder le fichier Parquet
output_path = "/home/jovyan/work/filtered_df_output.parquet"

# Enregistrez le DataFrame en Parquet
filtered_df.write.mode("overwrite").parquet(output_path)


# Afficher quelques lignes du DataFrame filtré
filtered_df.show(10, truncate=False)

In [ ]:
# filtered_df.count()

## 1. Configuration de la session Spark et chargement des données


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, lit, count, sum, avg, max as spark_max, datediff, when, countDistinct, row_number, round, dayofweek
from pyspark.sql.window import Window
import pandas as pd

### Créer une session Spark avec des configurations optimisées

In [2]:
spark = SparkSession.builder \
    .appName("E-commerce Amazing Analysis") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.driver.cores", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.2") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "4g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/28 09:23:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Lecture du fichier Parquet

In [3]:
output_path = "/home/jovyan/work/filtered_df_output.parquet"
filtered_df = spark.read.parquet(output_path)

### Conversion du champ event_time en type timestamp

In [4]:
filtered_df = filtered_df.withColumn("event_time", col("event_time").cast("timestamp"))

## 2. Création et chargement des mappings pour category_code et brand


### Extraire les paires uniques product_id, category_id et category_code

In [5]:
product_category_mapping_df = filtered_df.select("product_id", "category_id", "category_code").distinct()
brand_mapping_df = filtered_df.select("product_id", "brand").distinct()

### Sauvegarder ces mappings dans des fichiers Parquet pour une utilisation ultérieure

In [6]:
product_category_mapping_output_path = "/home/jovyan/work/product_category_mapping.parquet"
brand_mapping_output_path = "/home/jovyan/work/brand_mapping.parquet"
product_category_mapping_df.write.mode("overwrite").parquet(product_category_mapping_output_path)
brand_mapping_df.write.mode("overwrite").parquet(brand_mapping_output_path)

### Charger les mappings depuis les fichiers Parquet

In [7]:
product_category_mapping_df = spark.read.parquet(product_category_mapping_output_path)
brand_mapping_df = spark.read.parquet(brand_mapping_output_path)

### Renommer les colonnes dans les DataFrames de mapping pour éviter l'ambiguïté

In [8]:
product_category_mapping_df = product_category_mapping_df.withColumnRenamed("category_code", "mapped_category_code")
brand_mapping_df = brand_mapping_df.withColumnRenamed("brand", "mapped_brand")

### Joindre filtered_df avec product_category_mapping_df et brand_mapping_df pour ajouter les colonnes 'mapped_category_code' et 'mapped_brand'


In [9]:
filtered_df_with_mapping = filtered_df.join(product_category_mapping_df, on="product_id", how="left") \
                                      .join(brand_mapping_df, on="product_id", how="left")

### Remplacer les valeurs NULL dans 'category_code' et 'brand' par les valeurs correspondantes de la jointure


In [10]:
filtered_df = filtered_df_with_mapping.withColumn(
    "category_code",
    coalesce(filtered_df_with_mapping["category_code"], filtered_df_with_mapping["mapped_category_code"])
).withColumn(
    "brand",
    coalesce(filtered_df_with_mapping["brand"], filtered_df_with_mapping["mapped_brand"]))

## 3. Ajout de colonnes supplémentaires

### Ajout des colonnes supplémentaires

In [11]:
filtered_df = filtered_df.withColumn("event_day_of_week", dayofweek(col("event_time")))

### Définition des périodes de temps


In [12]:
now = filtered_df.select(spark_max("event_time")).collect()[0][0]
last_2_months = now - pd.DateOffset(months=2)
last_5_months = now - pd.DateOffset(months=5)
last_7_months = now - pd.DateOffset(months=7)


### Filtrage des données pour chaque période

In [13]:
filtered_df_2m = filtered_df.filter(col("event_time") >= lit(last_2_months))
filtered_df_5m = filtered_df.filter(col("event_time") >= lit(last_5_months))
filtered_df_7m = filtered_df.filter(col("event_time") >= lit(last_7_months))


## 4. Calcul des statistiques par utilisateur et période


### Fonction pour calculer les statistiques par utilisateur et périodes écoulées


In [14]:
def compute_user_stats(df, period):
    views = df.filter(col("event_type") == "view").groupBy("user_id").agg(count("*").alias(f"number_of_views_{period}"))
    carts = df.filter(col("event_type") == "cart").groupBy("user_id").agg(count("*").alias(f"number_of_carts_{period}"))
    sessions = df.groupBy("user_id").agg(countDistinct("user_session").alias(f"number_of_sessions_{period}"))
    purchases = df.filter(col("event_type") == "purchase").groupBy("user_id").agg(
        count("*").alias(f"count_products_{period}"),
        round(avg("price"), 2).alias(f"avg_price_{period}")
    )
    return views.join(carts, "user_id").join(sessions, "user_id").join(purchases, "user_id")



### Calcul des statistiques pour chaque période

In [15]:
stats_2m = compute_user_stats(filtered_df_2m, "2m")
stats_5m = compute_user_stats(filtered_df_5m, "5m")
stats_7m = compute_user_stats(filtered_df_7m, "7m")


### Union des statistiques pour les différentes périodes

In [16]:
stats_all = stats_2m.join(stats_5m, "user_id").join(stats_7m, "user_id")

## 5. Calcul des autres statistiques

### Calcul des autres statistiques

In [17]:
last_purchase = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(spark_max("event_time").alias("last_purchase"))
days_since_last_purchase = last_purchase.withColumn("days_since_last_purchase", datediff(lit(now), col("last_purchase")))

total_purchase_value = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(round(sum("price"), 2).alias("total_purchase_value"))

### Calcul des abandons de panier


In [18]:
cart_abandonments = filtered_df.groupBy("user_id").agg(
    (count(when(col("event_type") == "cart", True)) - count(when(col("event_type") == "purchase", True))).alias("cart_abandonments")
)

## 6. Calcul de la fidélité à une marque


### Calcul de la fidélité à une marque

In [19]:

total_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(count("*").alias("total_purchases"))
brand_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id", "brand").agg(count("*").alias("brand_purchases"))


### Déterminer la marque la plus achetée par chaque utilisateur


In [20]:
window_spec = Window.partitionBy("user_id").orderBy(col("brand_purchases").desc())
most_purchased_brand = brand_purchases_by_user.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1).drop("rank")


### Calcul de la fidélité

In [21]:
brand_loyalty = most_purchased_brand.join(total_purchases_by_user, "user_id").withColumn("brand_loyalty", round(col("brand_purchases") / col("total_purchases"), 2))


## 7. Détermination de la catégorie la plus achetée


### Déterminer la catégorie la plus achetée par chaque utilisateur

In [22]:
category_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id", "category_code").agg(count("*").alias("category_purchases"))
most_purchased_category = category_purchases_by_user.withColumn("rank", row_number().over(Window.partitionBy("user_id").orderBy(col("category_purchases").desc()))).filter(col("rank") == 1).drop("rank")


## 8. Jointure de toutes les statistiques et préparation des résultats finaux

### Joindre toutes les statistiques

In [23]:

user_stats_df = stats_all.join(days_since_last_purchase, "user_id") \
    .join(total_purchase_value, "user_id") \
    .join(cart_abandonments, "user_id") \
    .join(brand_loyalty.select("user_id", "brand_loyalty", "brand"), "user_id", "left") \
    .join(most_purchased_category.select("user_id", "category_code"), "user_id", "left") \
    .select("user_id", 
            "number_of_views_2m", "number_of_views_5m", "number_of_views_7m", 
            "number_of_carts_2m", "number_of_carts_5m", "number_of_carts_7m", 
            "number_of_sessions_2m", "number_of_sessions_5m", "number_of_sessions_7m", 
            "count_products_2m", "count_products_5m", "count_products_7m", 
            "avg_price_2m", "avg_price_5m", "avg_price_7m", 
            "total_purchase_value", "days_since_last_purchase", "cart_abandonments",
            "brand_loyalty", "brand", "category_code")


### Renommer les colonnes pour les préférences

In [24]:
user_stats_df = user_stats_df.withColumnRenamed("brand", "preferred_brand") \
                             .withColumnRenamed("category_code", "preferred_category")

### Remplacer les valeurs NULL dans 'preferred_brand' et 'preferred_category' par des valeurs par défaut

In [25]:
user_stats_df = user_stats_df.withColumn("preferred_brand", coalesce(col("preferred_brand"), lit("No Brand"))) \
                             .withColumn("preferred_category", coalesce(col("preferred_category"), lit("No Category")))

### Définir des segments d'utilisateurs basés sur la valeur totale des achats

In [26]:
user_stats_df = user_stats_df.withColumn("user_segment", 
                                         when(col("total_purchase_value") > 1000, "High").
                                         when((col("total_purchase_value") <= 1000) & (col("total_purchase_value") > 500), "Medium").
                                         otherwise("Low"))

### Affichage du DataFrame final

In [27]:
# user_stats_df.show()

## 9. Enregistrement et sauvegarde de user_stats_df

### Enregistrer et sauvegarder le DataFrame final dans un fichier Parquet

In [28]:
user_stats_output_path = "/home/jovyan/work/user_stats_df_output.parquet"
user_stats_df.write.mode("overwrite").parquet(user_stats_output_path)


# Analyse 

## Descriptive des colonnes



**user_id :** Identifiant unique de chaque utilisateur. 

**number_of_views_2m, number_of_views_5m, number_of_views_7m :**

**number_of_views_2m** : Nombre de fois qu'un utilisateur a consulté des produits au cours des 2 derniers mois.

**number_of_views_5m** : Nombre de fois qu'un utilisateur a consulté des produits au cours des 5 derniers mois.

**number_of_views_7m** : Nombre de fois qu'un utilisateur a consulté des produits au cours des 7 derniers mois.

**number_of_carts_2m, number_of_carts_5m, number_of_carts_7m :**

**number_of_carts_2m** : Nombre de produits ajoutés au panier par un utilisateur au cours des 2 derniers mois.

**number_of_carts_5m** : Nombre de produits ajoutés au panier par un utilisateur au cours des 5 derniers mois.

**number_of_carts_7m** : Nombre de produits ajoutés au panier par un utilisateur au cours des 7 derniers mois.

**number_of_sessions_2m, number_of_sessions_5m, number_of_sessions_7m :**

**number_of_sessions_2m**: Nombre de sessions de navigation d'un utilisateur au cours des 2 derniers mois.

**number_of_sessions_5m** : Nombre de sessions de navigation d'un utilisateur au cours des 5 derniers mois.

**number_of_sessions_7m** : Nombre de sessions de navigation d'un utilisateur au cours des 7 derniers mois.

**count_products_2m, count_products_5m, count_products_7m :**

**count_products_2m** : Nombre de produits achetés par un utilisateur au cours des 2 derniers mois.

**count_products_5m** : Nombre de produits achetés par un utilisateur au cours des 5 derniers mois.

**count_products_7m** : Nombre de produits achetés par un utilisateur au cours des 7 derniers mois.

**avg_price_2m, avg_price_5m, avg_price_7m :**

**avg_price_2m** : Prix moyen des produits achetés par un utilisateur au cours des 2 derniers mois.

**avg_price_5m**: Prix moyen des produits achetés par un utilisateur au cours des 5 derniers mois.

**avg_price_7m** : Prix moyen des produits achetés par un utilisateur au cours des 7 derniers mois.

**total_purchase_value** : Valeur totale des achats effectués par un utilisateur sur l'ensemble de la période considérée (oct 19 à avril 20 inclus).

**days_since_last_purchase** : Nombre de jours écoulés depuis le dernier achat de l'utilisateur.

**cart_abandonments**: Nombre de produits ajoutés au panier mais non achetés par un utilisateur. Cela mesure les abandons de panier, **indiquant des intentions d'achat non réalisées**.

**brand_loyalty** : Fidélité à une marque spécifique. Cette métrique est calculée comme la proportion des achats effectués auprès de la marque la plus fréquemment achetée par rapport au total des achats de l'utilisateur.

**preferred_brand** : Marque la plus fréquemment achetée par un utilisateur. S'il n'y a pas de données d'achat pour déterminer une marque préférée, la valeur par défaut est **"No Brand"**.

**preferred_category** : Catégorie de produit la plus fréquemment achetée par un utilisateur. S'il n'y a pas de données d'achat pour déterminer une catégorie préférée, la valeur par défaut est **"No Category"** .

**user_segment**: Segment d'utilisateur basé sur la valeur totale des achats. Il peut être catégorisé en :

- High : Utilisateurs dont la valeur totale des achats est supérieure à 1000.

- Medium : Utilisateurs dont la valeur totale des achats est entre 500 et 1000.
  
- Low : Utilisateurs dont la valeur totale des achats est inférieure à 500.

**avg_session_duration** : Combien de temps les utilisateurs passent sur le site, exrimés en minutes.

**most_active_day** : Préférences de l'utilisateur en termes de jours de navigation.

**most_active_time** : Utilisateurs en fonction de leurs habitudes de navigation quotidienne.


In [2]:
!pip install pandas pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 19.7 MB/s eta 0:00:0000:0100:01


In [3]:
import pandas as pd

# Définir le chemin du fichier Parquet
user_stats_input_path = "/home/jovyan/work/user_stats_df_output.parquet"

# Lire le fichier Parquet en utilisant pandas
user_stats_df = pd.read_parquet(user_stats_input_path, engine='pyarrow')

# Afficher les premières lignes du DataFrame pour vérifier la lecture
print(user_stats_df.head())


     user_id  number_of_views_2m  number_of_views_5m  number_of_views_7m  \
0  359242441                 148                 148                 148   
1  438263431                 253                 641                 655   
2  439490571                  58                  58                  58   
3  469107971                   5                   5                   5   
4  472393541                   8                  18                  18   

   number_of_carts_2m  number_of_carts_5m  number_of_carts_7m  \
0                   8                   8                   8   
1                   6                   8                   8   
2                   4                   4                   4   
3                   2                   2                   2   
4                   2                   2                   2   

   number_of_sessions_2m  number_of_sessions_5m  number_of_sessions_7m  ...  \
0                      9                      9                      9  .

In [6]:
user_stats_df.shape

(84864, 23)

In [11]:
user_stats_df['user_id'].nunique()

84864

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, lit, count, sum, avg, max as spark_max, min as spark_min, datediff, when, countDistinct, row_number, round, dayofweek, hour
from pyspark.sql.window import Window
import pandas as pd

# 1. Configuration de la session Spark et chargement des données
# Créer une session Spark avec des configurations optimisées
spark = SparkSession.builder \
    .appName("E-commerce Amazing Analysis") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.driver.cores", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.2") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "4g") \
    .getOrCreate()

# Définir le chemin du fichier Parquet
output_path = "/home/jovyan/work/filtered_df_output.parquet"

# Lire le fichier Parquet
filtered_df = spark.read.parquet(output_path)

# Conversion du champ event_time en type timestamp
filtered_df = filtered_df.withColumn("event_time", col("event_time").cast("timestamp"))

# 2. Création et chargement des mappings pour `category_code` et `brand`
# Extraire les paires uniques product_id, category_id et category_code
product_category_mapping_df = filtered_df.select("product_id", "category_id", "category_code").distinct()
brand_mapping_df = filtered_df.select("product_id", "brand").distinct()

# Sauvegarder ces mappings dans des fichiers Parquet pour une utilisation ultérieure
product_category_mapping_output_path = "/home/jovyan/work/product_category_mapping.parquet"
brand_mapping_output_path = "/home/jovyan/work/brand_mapping.parquet"
product_category_mapping_df.write.mode("overwrite").parquet(product_category_mapping_output_path)
brand_mapping_df.write.mode("overwrite").parquet(brand_mapping_output_path)

# Charger les mappings depuis les fichiers Parquet
product_category_mapping_df = spark.read.parquet(product_category_mapping_output_path)
brand_mapping_df = spark.read.parquet(brand_mapping_output_path)

# Renommer les colonnes dans les DataFrames de mapping pour éviter l'ambiguïté
product_category_mapping_df = product_category_mapping_df.withColumnRenamed("category_code", "mapped_category_code")
brand_mapping_df = brand_mapping_df.withColumnRenamed("brand", "mapped_brand")

# Joindre filtered_df avec product_category_mapping_df et brand_mapping_df pour ajouter les colonnes 'mapped_category_code' et 'mapped_brand'
filtered_df_with_mapping = filtered_df.join(product_category_mapping_df, on="product_id", how="left") \
                                      .join(brand_mapping_df, on="product_id", how="left")

# Remplacer les valeurs NULL dans 'category_code' et 'brand' par les valeurs correspondantes de la jointure
filtered_df = filtered_df_with_mapping.withColumn(
    "category_code",
    coalesce(filtered_df_with_mapping["category_code"], filtered_df_with_mapping["mapped_category_code"])
).withColumn(
    "brand",
    coalesce(filtered_df_with_mapping["brand"], filtered_df_with_mapping["mapped_brand"])
)

# 3. Ajout de colonnes supplémentaires
# Ajout des colonnes supplémentaires
filtered_df = filtered_df.withColumn("event_day_of_week", dayofweek(col("event_time"))) \
                         .withColumn("event_hour", hour(col("event_time")))

# Définition des périodes de temps
now = filtered_df.select(spark_max("event_time")).collect()[0][0]
last_2_months = now - pd.DateOffset(months=2)
last_5_months = now - pd.DateOffset(months=5)
last_7_months = now - pd.DateOffset(months=7)

# Filtrage des données pour chaque période
filtered_df_2m = filtered_df.filter(col("event_time") >= lit(last_2_months))
filtered_df_5m = filtered_df.filter(col("event_time") >= lit(last_5_months))
filtered_df_7m = filtered_df.filter(col("event_time") >= lit(last_7_months))

# 4. Calcul des statistiques par utilisateur et période
# Fonction pour calculer les statistiques par utilisateur et période
def compute_user_stats(df, period):
    views = df.filter(col("event_type") == "view").groupBy("user_id").agg(count("*").alias(f"number_of_views_{period}"))
    carts = df.filter(col("event_type") == "cart").groupBy("user_id").agg(count("*").alias(f"number_of_carts_{period}"))
    sessions = df.groupBy("user_id").agg(countDistinct("user_session").alias(f"number_of_sessions_{period}"))
    purchases = df.filter(col("event_type") == "purchase").groupBy("user_id").agg(
        count("*").alias(f"count_products_{period}"),
        round(avg("price"), 2).alias(f"avg_price_{period}")
    )
    return views.join(carts, "user_id").join(sessions, "user_id").join(purchases, "user_id")

# Calcul des statistiques pour chaque période
stats_2m = compute_user_stats(filtered_df_2m, "2m")
stats_5m = compute_user_stats(filtered_df_5m, "5m")
stats_7m = compute_user_stats(filtered_df_7m, "7m")

# Union des statistiques pour les différentes périodes
stats_all = stats_2m.join(stats_5m, "user_id").join(stats_7m, "user_id")

# 5. Calcul des autres statistiques
# Calcul des autres statistiques
last_purchase = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(spark_max("event_time").alias("last_purchase"))
days_since_last_purchase = last_purchase.withColumn("days_since_last_purchase", datediff(lit(now), col("last_purchase")))

total_purchase_value = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(round(sum("price"), 2).alias("total_purchase_value"))

# Calcul des abandons de panier
cart_abandonments = filtered_df.groupBy("user_id").agg(
    (count(when(col("event_type") == "cart", True)) - count(when(col("event_type") == "purchase", True))).alias("cart_abandonments")
)

# 6. Calcul de la fidélité à une marque
# Calcul de la fidélité à une marque
total_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(count("*").alias("total_purchases"))
brand_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id", "brand").agg(count("*").alias("brand_purchases"))

# Déterminer la marque la plus achetée par chaque utilisateur
window_spec = Window.partitionBy("user_id").orderBy(col("brand_purchases").desc())
most_purchased_brand = brand_purchases_by_user.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1).drop("rank")

# Calcul de la fidélité
brand_loyalty = most_purchased_brand.join(total_purchases_by_user, "user_id").withColumn("brand_loyalty", round(col("brand_purchases") / col("total_purchases"), 2))

# 7. Détermination de la catégorie la plus achetée
# Déterminer la catégorie la plus achetée par chaque utilisateur
category_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id", "category_code").agg(count("*").alias("category_purchases"))
most_purchased_category = category_purchases_by_user.withColumn("rank", row_number().over(Window.partitionBy("user_id").orderBy(col("category_purchases").desc()))).filter(col("rank") == 1).drop("rank")

# 8. Calcul du temps passé sur une session et du moment le plus actif
session_times = filtered_df.groupBy("user_id", "user_session").agg(
    (spark_max("event_time").cast("long") - spark_min("event_time").cast("long")).alias("session_duration")
).groupBy("user_id").agg(
    round(avg("session_duration") / 60, 2).alias("avg_session_duration")  # moyenne de la durée des sessions en minutes
)

most_active_day = filtered_df.groupBy("user_id", "event_day_of_week").agg(count("*").alias("day_activity")) \
                             .withColumn("rank", row_number().over(Window.partitionBy("user_id").orderBy(col("day_activity").desc()))) \
                             .filter(col("rank") == 1).drop("rank") \
                             .select("user_id", col("event_day_of_week").alias("most_active_day"))

most_active_time = filtered_df.withColumn("time_of_day",
    when(col("event_hour").between(5, 12), "morning")
    .when(col("event_hour").between(12, 17), "afternoon")
    .otherwise("evening")
).groupBy("user_id", "time_of_day").agg(count("*").alias("time_activity")) \
  .withColumn("rank", row_number().over(Window.partitionBy("user_id").orderBy(col("time_activity").desc()))) \
  .filter(col("rank") == 1).drop("rank") \
  .select("user_id", col("time_of_day").alias("most_active_time"))

# 9. Jointure de toutes les statistiques et préparation des résultats finaux
# Joindre toutes les statistiques
user_stats_df = stats_all.join(days_since_last_purchase, "user_id") \
    .join(total_purchase_value, "user_id") \
    .join(cart_abandonments, "user_id") \
    .join(brand_loyalty.select("user_id", "brand_loyalty", "brand"), "user_id", "left") \
    .join(most_purchased_category.select("user_id", "category_code"), "user_id", "left") \
    .join(session_times, "user_id", "left") \
    .join(most_active_day, "user_id", "left") \
    .join(most_active_time, "user_id", "left") \
    .select("user_id", 
            "number_of_views_2m", "number_of_views_5m", "number_of_views_7m", 
            "number_of_carts_2m", "number_of_carts_5m", "number_of_carts_7m", 
            "number_of_sessions_2m", "number_of_sessions_5m", "number_of_sessions_7m", 
            "count_products_2m", "count_products_5m", "count_products_7m", 
            "avg_price_2m", "avg_price_5m", "avg_price_7m", 
            "total_purchase_value", "days_since_last_purchase", "cart_abandonments",
            "brand_loyalty", "brand", "category_code", "avg_session_duration",
            "most_active_day", "most_active_time")

# Renommer les colonnes pour les préférences
user_stats_df = user_stats_df.withColumnRenamed("brand", "preferred_brand") \
                             .withColumnRenamed("category_code", "preferred_category")

# Remplacer les valeurs NULL dans 'preferred_brand' et 'preferred_category' par des valeurs par défaut
user_stats_df = user_stats_df.withColumn("preferred_brand", coalesce(col("preferred_brand"), lit("No Brand"))) \
                             .withColumn("preferred_category", coalesce(col("preferred_category"), lit("No Category")))

# Définir des segments d'utilisateurs basés sur la valeur totale des achats
user_stats_df = user_stats_df.withColumn("user_segment", 
                                         when(col("total_purchase_value") > 1000, "High").
                                         when((col("total_purchase_value") <= 1000) & (col("total_purchase_value") > 500), "Medium").
                                         otherwise("Low"))

# Affichage du DataFrame final
user_stats_df.show()

# 10. Enregistrement et sauvegarde de `user_stats_df`
# Définir le chemin de sortie pour le DataFrame final
user_stats_output_path = "/home/jovyan/work/user_stats_df_output.parquet"

# Enregistrer et sauvegarder le DataFrame final dans un fichier Parquet
user_stats_df.write.mode("overwrite").parquet(user_stats_output_path)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/29 08:47:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/29 08:48:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+------------------+------------------+------------------+------------------+------------------+------------------+---------------------+---------------------+---------------------+-----------------+-----------------+-----------------+------------+------------+------------+--------------------+------------------------+-----------------+-------------+---------------+--------------------+--------------------+---------------+----------------+------------+
|  user_id|number_of_views_2m|number_of_views_5m|number_of_views_7m|number_of_carts_2m|number_of_carts_5m|number_of_carts_7m|number_of_sessions_2m|number_of_sessions_5m|number_of_sessions_7m|count_products_2m|count_products_5m|count_products_7m|avg_price_2m|avg_price_5m|avg_price_7m|total_purchase_value|days_since_last_purchase|cart_abandonments|brand_loyalty|preferred_brand|  preferred_category|avg_session_duration|most_active_day|most_active_time|user_segment|
+---------+------------------+------------------+-----------------

In [2]:
spark.stop()

ConnectionRefusedError: [Errno 111] Connection refused

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, lit, count, sum, avg, max as spark_max, min as spark_min, datediff, when, countDistinct, row_number, round, dayofweek, hour
from pyspark.sql.window import Window
import pandas as pd

# 1. Configuration de la session Spark et chargement des données
# Créer une session Spark avec des configurations optimisées
spark = SparkSession.builder \
    .appName("E-commerce Amazing Analysis") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.driver.cores", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.2") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "4g") \
    .getOrCreate()

# Définir le chemin du fichier Parquet
output_path = "/home/jovyan/work/filtered_df_output.parquet"

# Lire le fichier Parquet
filtered_df = spark.read.parquet(output_path)

# Conversion du champ event_time en type timestamp
filtered_df = filtered_df.withColumn("event_time", col("event_time").cast("timestamp"))

# 2. Création et chargement des mappings pour `category_code` et `brand`
# Extraire les paires uniques product_id, category_id et category_code
product_category_mapping_df = filtered_df.select("product_id", "category_id", "category_code").distinct()
brand_mapping_df = filtered_df.select("product_id", "brand").distinct()

# Sauvegarder ces mappings dans des fichiers Parquet pour une utilisation ultérieure
product_category_mapping_output_path = "/home/jovyan/work/product_category_mapping.parquet"
brand_mapping_output_path = "/home/jovyan/work/brand_mapping.parquet"
product_category_mapping_df.write.mode("overwrite").parquet(product_category_mapping_output_path)
brand_mapping_df.write.mode("overwrite").parquet(brand_mapping_output_path)

# Charger les mappings depuis les fichiers Parquet
product_category_mapping_df = spark.read.parquet(product_category_mapping_output_path)
brand_mapping_df = spark.read.parquet(brand_mapping_output_path)

# Renommer les colonnes dans les DataFrames de mapping pour éviter l'ambiguïté
product_category_mapping_df = product_category_mapping_df.withColumnRenamed("category_code", "mapped_category_code")
brand_mapping_df = brand_mapping_df.withColumnRenamed("brand", "mapped_brand")

# Joindre filtered_df avec product_category_mapping_df et brand_mapping_df pour ajouter les colonnes 'mapped_category_code' et 'mapped_brand'
filtered_df_with_mapping = filtered_df.join(product_category_mapping_df, on="product_id", how="left") \
                                      .join(brand_mapping_df, on="product_id", how="left")

# Remplacer les valeurs NULL dans 'category_code' et 'brand' par les valeurs correspondantes de la jointure
filtered_df = filtered_df_with_mapping.withColumn(
    "category_code",
    coalesce(filtered_df_with_mapping["category_code"], filtered_df_with_mapping["mapped_category_code"])
).withColumn(
    "brand",
    coalesce(filtered_df_with_mapping["brand"], filtered_df_with_mapping["mapped_brand"])
)

# 3. Ajout de colonnes supplémentaires
# Ajout des colonnes supplémentaires
filtered_df = filtered_df.withColumn("event_day_of_week", dayofweek(col("event_time"))) \
                         .withColumn("event_hour", hour(col("event_time")))

# Définition des périodes de temps
now = filtered_df.select(spark_max("event_time")).collect()[0][0]
last_2_months = now - pd.DateOffset(months=2)
last_5_months = now - pd.DateOffset(months=5)
last_7_months = now - pd.DateOffset(months=7)

# Filtrage des données pour chaque période
filtered_df_2m = filtered_df.filter(col("event_time") >= lit(last_2_months))
filtered_df_5m = filtered_df.filter(col("event_time") >= lit(last_5_months))
filtered_df_7m = filtered_df.filter(col("event_time") >= lit(last_7_months))

# 4. Calcul des statistiques par utilisateur et période
# Fonction pour calculer les statistiques par utilisateur et période
def compute_user_stats(df, period):
    views = df.filter(col("event_type") == "view").groupBy("user_id").agg(count("*").alias(f"number_of_views_{period}"))
    carts = df.filter(col("event_type") == "cart").groupBy("user_id").agg(count("*").alias(f"number_of_carts_{period}"))
    sessions = df.groupBy("user_id").agg(countDistinct("user_session").alias(f"number_of_sessions_{period}"))
    purchases = df.filter(col("event_type") == "purchase").groupBy("user_id").agg(
        count("*").alias(f"count_products_{period}"),
        round(avg("price"), 2).alias(f"avg_price_{period}")
    )
    return views.join(carts, "user_id").join(sessions, "user_id").join(purchases, "user_id")

# Calcul des statistiques pour chaque période
stats_2m = compute_user_stats(filtered_df_2m, "2m")
stats_5m = compute_user_stats(filtered_df_5m, "5m")
stats_7m = compute_user_stats(filtered_df_7m, "7m")

# Union des statistiques pour les différentes périodes
stats_all = stats_2m.join(stats_5m, "user_id").join(stats_7m, "user_id")

# 5. Calcul des autres statistiques
# Calcul des autres statistiques
last_purchase = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(spark_max("event_time").alias("last_purchase"))
days_since_last_purchase = last_purchase.withColumn("days_since_last_purchase", datediff(lit(now), col("last_purchase")))

total_purchase_value = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(round(sum("price"), 2).alias("total_purchase_value"))

# Calcul des abandons de panier
cart_events = filtered_df.filter(col("event_type") == "cart").groupBy("user_id").agg(count("*").alias("cart_count"))
purchase_events = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(count("*").alias("purchase_count"))

cart_abandonments = cart_events.join(purchase_events, "user_id", "left") \
                               .withColumn("cart_abandonments", when(col("cart_count") > col("purchase_count"), col("cart_count") - col("purchase_count")).otherwise(0)) \
                               .select("user_id", "cart_abandonments")

# 6. Calcul de la fidélité à une marque
# Calcul de la fidélité à une marque
total_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(count("*").alias("total_purchases"))
brand_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id", "brand").agg(count("*").alias("brand_purchases"))

# Déterminer la marque la plus achetée par chaque utilisateur
window_spec = Window.partitionBy("user_id").orderBy(col("brand_purchases").desc())
most_purchased_brand = brand_purchases_by_user.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1).drop("rank")

# Calcul de la fidélité
brand_loyalty = most_purchased_brand.join(total_purchases_by_user, "user_id").withColumn("brand_loyalty", round(col("brand_purchases") / col("total_purchases"), 2))

# 7. Détermination de la catégorie la plus achetée
# Déterminer la catégorie la plus achetée par chaque utilisateur
category_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id", "category_code").agg(count("*").alias("category_purchases"))
most_purchased_category = category_purchases_by_user.withColumn("rank", row_number().over(Window.partitionBy("user_id").orderBy(col("category_purchases").desc()))).filter(col("rank") == 1).drop("rank")

# 8. Calcul du temps passé sur une session et du moment le plus actif
session_times = filtered_df.groupBy("user_id", "user_session").agg(
    (spark_max("event_time").cast("long") - spark_min("event_time").cast("long")).alias("session_duration")
).groupBy("user_id").agg(
    round(avg("session_duration") / 60, 2).alias("avg_session_duration")  # moyenne de la durée des sessions en minutes
)

most_active_day = filtered_df.groupBy("user_id", "event_day_of_week").agg(count("*").alias("day_activity")) \
                             .withColumn("rank", row_number().over(Window.partitionBy("user_id").orderBy(col("day_activity").desc()))) \
                             .filter(col("rank") == 1).drop("rank") \
                             .select("user_id", col("event_day_of_week").alias("most_active_day"))

most_active_time = filtered_df.withColumn("time_of_day",
    when(col("event_hour").between(5, 12), "morning")
    .when(col("event_hour").between(12, 17), "afternoon")
    .otherwise("evening")
).groupBy("user_id", "time_of_day").agg(count("*").alias("time_activity")) \
  .withColumn("rank", row_number().over(Window.partitionBy("user_id").orderBy(col("time_activity").desc()))) \
  .filter(col("rank") == 1).drop("rank") \
  .select("user_id", col("time_of_day").alias("most_active_time"))

# 9. Jointure de toutes les statistiques et préparation des résultats finaux
# Joindre toutes les statistiques
user_stats_df = stats_all.join(days_since_last_purchase, "user_id") \
    .join(total_purchase_value, "user_id") \
    .join(cart_abandonments, "user_id") \
    .join(brand_loyalty.select("user_id", "brand_loyalty", "brand"), "user_id", "left") \
    .join(most_purchased_category.select("user_id", "category_code"), "user_id", "left") \
    .join(session_times, "user_id", "left") \
    .join(most_active_day, "user_id", "left") \
    .join(most_active_time, "user_id", "left") \
    .select("user_id", 
            "number_of_views_2m", "number_of_views_5m", "number_of_views_7m", 
            "number_of_carts_2m", "number_of_carts_5m", "number_of_carts_7m", 
            "number_of_sessions_2m", "number_of_sessions_5m", "number_of_sessions_7m", 
            "count_products_2m", "count_products_5m", "count_products_7m", 
            "avg_price_2m", "avg_price_5m", "avg_price_7m", 
            "total_purchase_value", "days_since_last_purchase", "cart_abandonments",
            "brand_loyalty", "brand", "category_code", "avg_session_duration",
            "most_active_day", "most_active_time")

# Renommer les colonnes pour les préférences
user_stats_df = user_stats_df.withColumnRenamed("brand", "preferred_brand") \
                             .withColumnRenamed("category_code", "preferred_category")

# Remplacer les valeurs NULL dans 'preferred_brand' et 'preferred_category' par des valeurs par défaut
user_stats_df = user_stats_df.withColumn("preferred_brand", coalesce(col("preferred_brand"), lit("No Brand"))) \
                             .withColumn("preferred_category", coalesce(col("preferred_category"), lit("No Category")))

# Définir des segments d'utilisateurs basés sur la valeur totale des achats
user_stats_df = user_stats_df.withColumn("user_segment", 
                                         when(col("total_purchase_value") > 1000, "High").
                                         when((col("total_purchase_value") <= 1000) & (col("total_purchase_value") > 500), "Medium").
                                         otherwise("Low"))

# Affichage du DataFrame final
user_stats_df.show()

# 10. Enregistrement et sauvegarde de `user_stats_df`
# Définir le chemin de sortie pour le DataFrame final
user_stats_output_path = "/home/jovyan/work/user_stats_df_output.parquet"

# Enregistrer et sauvegarder le DataFrame final dans un fichier Parquet
user_stats_df.write.mode("overwrite").parquet(user_stats_output_path)


ERROR:root:Exception while sending command.                       (10 + 2) / 21]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: reentrant call inside <_io.BufferedReader name=59>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py

Py4JError: An error occurred while calling o718.parquet

# Vérification valeur négative

In [4]:
# Vérification des valeurs nulles par lots
def check_columns_for_nulls(df, columns):
    for column in columns:
        null_count = df.filter(col(column).isNull()).count()
        print(f"Column '{column}' has {null_count} null values.")

In [5]:
def check_columns_for_negatives(df, columns):
    for column in columns:
        negative_count = df.filter(col(column) < 0).count()
        print(f"Column '{column}' has {negative_count} negative values.")

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, lit, count, sum, avg, max as spark_max, min as spark_min, datediff, when, countDistinct, row_number, round, dayofweek, hour
from pyspark.sql.window import Window
import pandas as pd

# 1. Configuration de la session Spark et chargement des données
# Créer une session Spark avec des configurations optimisées pour la mémoire
spark = SparkSession.builder \
    .appName("E-commerce Amazing Analysis") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.instances", "4") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.2") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "8g") \
    .getOrCreate()

# Définir le chemin du fichier Parquet
output_path = "/home/jovyan/work/filtered_df_output.parquet"

# Lire le fichier Parquet
filtered_df = spark.read.parquet(output_path).cache()

# Conversion du champ event_time en type timestamp
filtered_df = filtered_df.withColumn("event_time", col("event_time").cast("timestamp"))

# 2. Création et chargement des mappings pour `category_code` et `brand`
# Extraire les paires uniques product_id, category_id et category_code
product_category_mapping_df = filtered_df.select("product_id", "category_id", "category_code").distinct().cache()
brand_mapping_df = filtered_df.select("product_id", "brand").distinct().cache()

# Sauvegarder ces mappings dans des fichiers Parquet pour une utilisation ultérieure
product_category_mapping_output_path = "/home/jovyan/work/product_category_mapping.parquet"
brand_mapping_output_path = "/home/jovyan/work/brand_mapping.parquet"
product_category_mapping_df.write.mode("overwrite").parquet(product_category_mapping_output_path)
brand_mapping_df.write.mode("overwrite").parquet(brand_mapping_output_path)

# Charger les mappings depuis les fichiers Parquet
product_category_mapping_df = spark.read.parquet(product_category_mapping_output_path).cache()
brand_mapping_df = spark.read.parquet(brand_mapping_output_path).cache()

# Renommer les colonnes dans les DataFrames de mapping pour éviter l'ambiguïté
product_category_mapping_df = product_category_mapping_df.withColumnRenamed("category_code", "mapped_category_code")
brand_mapping_df = brand_mapping_df.withColumnRenamed("brand", "mapped_brand")

# Joindre filtered_df avec product_category_mapping_df et brand_mapping_df pour ajouter les colonnes 'mapped_category_code' et 'mapped_brand'
filtered_df_with_mapping = filtered_df.join(product_category_mapping_df, on="product_id", how="left") \
                                      .join(brand_mapping_df, on="product_id", how="left")

# Remplacer les valeurs NULL dans 'category_code' et 'brand' par les valeurs correspondantes de la jointure
filtered_df = filtered_df_with_mapping.withColumn(
    "category_code",
    coalesce(filtered_df_with_mapping["category_code"], filtered_df_with_mapping["mapped_category_code"])
).withColumn(
    "brand",
    coalesce(filtered_df_with_mapping["brand"], filtered_df_with_mapping["mapped_brand"])
).cache()

# 3. Ajout de colonnes supplémentaires
# Ajout des colonnes supplémentaires
filtered_df = filtered_df.withColumn("event_day_of_week", dayofweek(col("event_time"))) \
                         .withColumn("event_hour", hour(col("event_time")))

# Définition des périodes de temps
now = filtered_df.select(spark_max("event_time")).collect()[0][0]
last_2_months = now - pd.DateOffset(months=2)
last_5_months = now - pd.DateOffset(months=5)
last_7_months = now - pd.DateOffset(months=7)

# Filtrage des données pour chaque période
filtered_df_2m = filtered_df.filter(col("event_time") >= lit(last_2_months)).cache()
filtered_df_5m = filtered_df.filter(col("event_time") >= lit(last_5_months)).cache()
filtered_df_7m = filtered_df.filter(col("event_time") >= lit(last_7_months)).cache()

# 4. Calcul des statistiques par utilisateur et période
# Fonction pour calculer les statistiques par utilisateur et période
def compute_user_stats(df, period):
    views = df.filter(col("event_type") == "view").groupBy("user_id").agg(count("*").alias(f"number_of_views_{period}"))
    carts = df.filter(col("event_type") == "cart").groupBy("user_id").agg(count("*").alias(f"number_of_carts_{period}"))
    sessions = df.groupBy("user_id").agg(countDistinct("user_session").alias(f"number_of_sessions_{period}"))
    purchases = df.filter(col("event_type") == "purchase").groupBy("user_id").agg(
        count("*").alias(f"count_products_{period}"),
        round(avg("price"), 2).alias(f"avg_price_{period}")
    )
    return views.join(carts, "user_id").join(sessions, "user_id").join(purchases, "user_id")

# Calcul des statistiques pour chaque période
stats_2m = compute_user_stats(filtered_df_2m, "2m").cache()
stats_5m = compute_user_stats(filtered_df_5m, "5m").cache()
stats_7m = compute_user_stats(filtered_df_7m, "7m").cache()

# Union des statistiques pour les différentes périodes
stats_all = stats_2m.join(stats_5m, "user_id").join(stats_7m, "user_id").cache()

# 5. Calcul des autres statistiques
# Calcul des autres statistiques
last_purchase = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(spark_max("event_time").alias("last_purchase")).cache()
days_since_last_purchase = last_purchase.withColumn("days_since_last_purchase", datediff(lit(now), col("last_purchase"))).cache()

total_purchase_value = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(round(sum("price"), 2).alias("total_purchase_value")).cache()

# Calcul des abandons de panier
cart_events = filtered_df.filter(col("event_type") == "cart").groupBy("user_id").agg(count("*").alias("cart_count")).cache()
purchase_events = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(count("*").alias("purchase_count")).cache()

cart_abandonments = cart_events.join(purchase_events, "user_id", "left") \
                               .withColumn("cart_abandonments", when(col("cart_count") > col("purchase_count"), col("cart_count") - col("purchase_count")).otherwise(0)) \
                               .select("user_id", "cart_abandonments").cache()

# 6. Calcul de la fidélité à une marque
# Calcul de la fidélité à une marque
total_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id").agg(count("*").alias("total_purchases")).cache()
brand_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id", "brand").agg(count("*").alias("brand_purchases")).cache()

# Déterminer la marque la plus achetée par chaque utilisateur
window_spec = Window.partitionBy("user_id").orderBy(col("brand_purchases").desc())
most_purchased_brand = brand_purchases_by_user.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1).drop("rank").cache()

# Calcul de la fidélité
brand_loyalty = most_purchased_brand.join(total_purchases_by_user, "user_id").withColumn("brand_loyalty", round(col("brand_purchases") / col("total_purchases"), 2)).cache()

# 7. Détermination de la catégorie la plus achetée
# Déterminer la catégorie la plus achetée par chaque utilisateur
category_purchases_by_user = filtered_df.filter(col("event_type") == "purchase").groupBy("user_id", "category_code").agg(count("*").alias("category_purchases")).cache()
most_purchased_category = category_purchases_by_user.withColumn("rank", row_number().over(Window.partitionBy("user_id").orderBy(col("category_purchases").desc()))).filter(col("rank") == 1).drop("rank").cache()

# 8. Calcul du temps passé sur une session et du moment le plus actif
session_times = filtered_df.groupBy("user_id", "user_session").agg(
    (spark_max("event_time").cast("long") - spark_min("event_time").cast("long")).alias("session_duration")
).groupBy("user_id").agg(
    round(avg("session_duration") / 60, 2).alias("avg_session_duration")  # moyenne de la durée des sessions en minutes
).cache()

most_active_day = filtered_df.groupBy("user_id", "event_day_of_week").agg(count("*").alias("day_activity")) \
                             .withColumn("rank", row_number().over(Window.partitionBy("user_id").orderBy(col("day_activity").desc()))) \
                             .filter(col("rank") == 1).drop("rank") \
                             .select("user_id", col("event_day_of_week").alias("most_active_day")).cache()

most_active_time = filtered_df.withColumn("time_of_day",
    when(col("event_hour").between(5, 12), "morning")
    .when(col("event_hour").between(12, 17), "afternoon")
    .otherwise("evening")
).groupBy("user_id", "time_of_day").agg(count("*").alias("time_activity")) \
  .withColumn("rank", row_number().over(Window.partitionBy("user_id").orderBy(col("time_activity").desc()))) \
  .filter(col("rank") == 1).drop("rank") \
  .select("user_id", col("time_of_day").alias("most_active_time")).cache()

# 9. Jointure de toutes les statistiques et préparation des résultats finaux
# Joindre toutes les statistiques
user_stats_df = stats_all.join(days_since_last_purchase, "user_id") \
    .join(total_purchase_value, "user_id") \
    .join(cart_abandonments, "user_id") \
    .join(brand_loyalty.select("user_id", "brand_loyalty", "brand"), "user_id", "left") \
    .join(most_purchased_category.select("user_id", "category_code"), "user_id", "left") \
    .join(session_times, "user_id", "left") \
    .join(most_active_day, "user_id", "left") \
    .join(most_active_time, "user_id", "left") \
    .select("user_id", 
            "number_of_views_2m", "number_of_views_5m", "number_of_views_7m", 
            "number_of_carts_2m", "number_of_carts_5m", "number_of_carts_7m", 
            "number_of_sessions_2m", "number_of_sessions_5m", "number_of_sessions_7m", 
            "count_products_2m", "count_products_5m", "count_products_7m", 
            "avg_price_2m", "avg_price_5m", "avg_price_7m", 
            "total_purchase_value", "days_since_last_purchase", "cart_abandonments",
            "brand_loyalty", "brand", "category_code", "avg_session_duration",
            "most_active_day", "most_active_time").cache()

# Renommer les colonnes pour les préférences
user_stats_df = user_stats_df.withColumnRenamed("brand", "preferred_brand") \
                             .withColumnRenamed("category_code", "preferred_category")

# Remplacer les valeurs NULL dans 'preferred_brand' et 'preferred_category' par des valeurs par défaut
user_stats_df = user_stats_df.withColumn("preferred_brand", coalesce(col("preferred_brand"), lit("No Brand"))) \
                             .withColumn("preferred_category", coalesce(col("preferred_category"), lit("No Category")))

# Définir des segments d'utilisateurs basés sur la valeur totale des achats
user_stats_df = user_stats_df.withColumn("user_segment", 
                                         when(col("total_purchase_value") > 1000, "High").
                                         when((col("total_purchase_value") <= 1000) & (col("total_purchase_value") > 500), "Medium").
                                         otherwise("Low")).cache()

# Affichage du DataFrame final
user_stats_df.show()

# 10. Vérification des colonnes pour des valeurs négatives
columns_to_check = [
    "number_of_views_2m", "number_of_views_5m", "number_of_views_7m",
    "number_of_carts_2m", "number_of_carts_5m", "number_of_carts_7m",
    "number_of_sessions_2m", "number_of_sessions_5m", "number_of_sessions_7m",
    "count_products_2m", "count_products_5m", "count_products_7m",
    "avg_price_2m", "avg_price_5m", "avg_price_7m",
    "total_purchase_value", "days_since_last_purchase", "cart_abandonments",
    "brand_loyalty", "avg_session_duration"
]



columns_batch_1 = ["number_of_views_2m", "number_of_views_5m", "number_of_views_7m"]
columns_batch_2 = ["number_of_carts_2m", "number_of_carts_5m", "number_of_carts_7m"]
columns_batch_3 = ["number_of_sessions_2m", "number_of_sessions_5m", "number_of_sessions_7m"]
columns_batch_4 = ["count_products_2m", "count_products_5m", "count_products_7m"]
columns_batch_5 = ["avg_price_2m", "avg_price_5m", "avg_price_7m", "total_purchase_value"]
columns_batch_6 = ["days_since_last_purchase", "cart_abandonments", "brand_loyalty", "avg_session_duration"]

check_columns_for_nulls(user_stats_df, columns_batch_1)
check_columns_for_nulls(user_stats_df, columns_batch_2)
check_columns_for_nulls(user_stats_df, columns_batch_3)
check_columns_for_nulls(user_stats_df, columns_batch_4)
check_columns_for_nulls(user_stats_df, columns_batch_5)
check_columns_for_nulls(user_stats_df, columns_batch_6)



check_columns_for_negatives(user_stats_df, columns_batch_1)
check_columns_for_negatives(user_stats_df, columns_batch_2)
check_columns_for_negatives(user_stats_df, columns_batch_3)
check_columns_for_negatives(user_stats_df, columns_batch_4)
check_columns_for_negatives(user_stats_df, columns_batch_5)
check_columns_for_negatives(user_stats_df, columns_batch_6)

# 11. Enregistrement et sauvegarde de `user_stats_df`
# Définir le chemin de sortie pour le DataFrame final
user_stats_output_path = "/home/jovyan/work/user_stats_df_output.parquet"

# Enregistrer et sauvegarder le DataFrame final dans un fichier Parquet
user_stats_df.write.mode("overwrite").parquet(user_stats_output_path)


# 11. Enregistrement et sauvegarde de `user_stats_df`
# Définir le chemin de sortie pour le DataFrame final
# user_stats_output_path = "/home/jovyan/work/user_stats_df_output.parquet"

# Enregistrer et sauvegarder le DataFrame final dans un fichier Parquet
# user_stats_df.write.mode("overwrite").parquet(user_stats_output_path)


ConnectionRefusedError: [Errno 111] Connection refused